In [5]:
import numpy as np
from syft.core.tensor.autodp.phi_tensor import PhiTensor
from syft.core.tensor.autodp.gamma_tensor import GammaTensor
from syft.core.tensor.fixed_precision_tensor import FixedPrecisionTensor as FPT
from syft.core.adp.data_subject_list import DataSubjectList
from syft.core.tensor.lazy_repeat_array import lazyrepeatarray as lra

## Torch Dataset equivalent

We have to implement:
- padding -> DONE!
- random horizontal flip -> DONE!
- random vertical flip -> DONE!
- random rotation -> Done but not confident about resultant LRA, or what to do with 0 values. -> Might try training the model without random rotations to see how testing accuracy suffers.
- normalize -> DONE!

In [6]:
a = PhiTensor(child=np.random.random((5,5)),data_subjects=np.ones((5,5)), min_vals=0, max_vals=1)

In [7]:
a

PhiTensor(child=FixedPrecisionTensor(child=[[35364 22056 14629 24277 31407]
 [11692 45525  3860 64992 41154]
 [ 7451 15905  7127 48296 34807]
 [14745 61587 49996 41563 54958]
 [63873 51892 39111 37338 32073]]), min_vals=<lazyrepeatarray data: 0 -> shape: (5, 5)>, max_vals=<lazyrepeatarray data: 1 -> shape: (5, 5)>)

In [8]:
a.pad(width=1)

PhiTensor(child=FixedPrecisionTensor(child=[[45525 11692 45525  3860 64992 41154 64992]
 [22056 35364 22056 14629 24277 31407 24277]
 [45525 11692 45525  3860 64992 41154 64992]
 [15905  7451 15905  7127 48296 34807 48296]
 [61587 14745 61587 49996 41563 54958 41563]
 [51892 63873 51892 39111 37338 32073 37338]
 [61587 14745 61587 49996 41563 54958 41563]]), min_vals=<lazyrepeatarray data: 0 -> shape: (7, 7)>, max_vals=<lazyrepeatarray data: 1 -> shape: (7, 7)>)

In [9]:
a.random_horizontal_flip()

PhiTensor(child=FixedPrecisionTensor(child=[[35364 22056 14629 24277 31407]
 [11692 45525  3860 64992 41154]
 [ 7451 15905  7127 48296 34807]
 [14745 61587 49996 41563 54958]
 [63873 51892 39111 37338 32073]]), min_vals=<lazyrepeatarray data: 0 -> shape: (5, 5)>, max_vals=<lazyrepeatarray data: 1 -> shape: (5, 5)>)

In [10]:
a.random_vertical_flip()

PhiTensor(child=FixedPrecisionTensor(child=[[63873 51892 39111 37338 32073]
 [14745 61587 49996 41563 54958]
 [ 7451 15905  7127 48296 34807]
 [11692 45525  3860 64992 41154]
 [35364 22056 14629 24277 31407]]), min_vals=<lazyrepeatarray data: 0 -> shape: (5, 5)>, max_vals=<lazyrepeatarray data: 1 -> shape: (5, 5)>)

In [11]:
a.random_rotation(degrees=20)

PhiTensor(child=FixedPrecisionTensor(child=[[    0     0     0     0     0     0]
 [    0 15518 38283 12735 27378     0]
 [    0 11753 11166 19484 64343     0]
 [    0 40891 42166 35037 41630     0]
 [    0 53974 46353 39784 53730     0]
 [    0     0     0     0     0     0]]), min_vals=<lazyrepeatarray data: 0 -> shape: (5, 5)>, max_vals=<lazyrepeatarray data: 1 -> shape: (5, 5)>)

In [12]:
a.normalize(mean=0.5, std=0.5)

PhiTensor(child=FixedPrecisionTensor(child=[[  5192 -21424 -36278 -16982  -2722]
 [-42152  25514 -57816  64448  16772]
 [-50634 -33726 -51282  31056   4078]
 [-36046  57638  34456  17590  44380]
 [ 62210  38248  12686   9140  -1390]]), min_vals=<lazyrepeatarray data: -1.0 -> shape: ()>, max_vals=<lazyrepeatarray data: 1.0 -> shape: ()>)

In [13]:
a.child.decode()

array([[0.53961182, 0.33654785, 0.22322083, 0.37043762, 0.47923279],
       [0.17840576, 0.69465637, 0.05889893, 0.99169922, 0.62796021],
       [0.11369324, 0.24269104, 0.10874939, 0.73693848, 0.53111267],
       [0.22499084, 0.93974304, 0.76287842, 0.63420105, 0.83859253],
       [0.97462463, 0.79180908, 0.5967865 , 0.56973267, 0.48939514]])

<hr>
<hr>

## Model Training


To Do:
- Implement the layers needed for a ConvNet for DP Tensors:
    - Conv layer
    - BatchNorm2D
    - LeakyReLU
    - AvgPool2d
    - Linear
- Do 1 forward pass with these layers
- Do 1 backprop with these layers

In [14]:
from syft.core.tensor.nn.conv_layers import Conv2d
import torch

In [15]:
import cv2

# Update image path below
img_path = "/home/shubham/PySyft/notebooks/medical-federated-learning-program/network-operators/archive/10253/0/10253_idx5_x1051_y651_class0.png"
img = cv2.imread(img_path)
img = cv2.resize(img, (50, 50))

In [16]:
dp_tensor = PhiTensor(child=img, data_subjects=[0]  , min_vals=0, max_vals=255)

In [17]:
r = Conv2d(dp_tensor, in_channels=3, out_channels=32, kernel_size=3, padding=2)

In [18]:
from syft.core.tensor.nn.batch_norm import BatchNorm2d

In [19]:
img.shape

(50, 50, 3)

In [20]:
torch.nn.BatchNorm2d(3)(torch.Tensor(img.reshape(1, *img.shape[::-1]))).shape

torch.Size([1, 3, 50, 50])

In [21]:
dpt2 = dp_tensor.copy()
dpt2.child = dp_tensor.child.reshape(1, *dp_tensor.shape[::-1])
BatchNorm2d(dpt2, 3).child.decode().shape

(1, 3, 50, 50)

In [22]:
torch.nn.BatchNorm2d(3)(torch.Tensor(img.reshape(1, *img.shape[::-1])))

tensor([[[[ 0.0870, -1.5196,  1.0985,  ...,  1.2473,  0.1762, -1.1923],
          [ 1.0985,  0.5927, -0.8353,  ..., -1.7874, -0.3891, -0.2998],
          [-1.8469, -0.3593,  0.2952,  ...,  0.0572, -1.7279, -0.0023],
          ...,
          [-1.4304,  0.1167,  0.6523,  ...,  0.1762, -1.2816, -0.1808],
          [ 0.5332, -0.8651,  1.1580,  ...,  1.2770,  0.7415, -0.4188],
          [ 1.2473,  0.8308, -0.2105,  ...,  0.5035,  1.3960,  0.4440]],

         [[-0.8683,  0.8349,  0.5916,  ..., -0.0471, -1.5982,  0.1354],
          [ 0.2875, -1.1724,  0.6524,  ...,  1.4128,  0.0746, -1.5982],
          [ 0.9870,  0.4091, -1.0508,  ..., -1.2028,  1.1086,  0.5308],
          ...,
          [ 1.2303, -0.0471, -1.6286,  ..., -0.2904,  1.2303, -0.3208],
          [-1.7199, -0.3208, -0.0775,  ...,  0.1354, -1.3549,  1.0478],
          [ 0.4395, -0.8683,  1.2607,  ...,  0.9566, -0.0775, -1.8415]],

         [[ 0.9140,  0.1708, -1.1032,  ...,  0.3300,  1.0201,  1.1528],
          [ 0.5424,  1.2059,  

In [23]:
BatchNorm2d(dpt2, 3).child.decode()

array([[[[ 0.08695984, -1.51960754,  1.09851074, ...,  1.24726868,
           0.17622375, -1.19233704],
         [ 1.09851074,  0.59274292, -0.83532715, ..., -1.78736877,
          -0.38905334, -0.29978943],
         [-1.84687805, -0.35929871,  0.29522705, ...,  0.0572052 ,
          -1.72787476, -0.00227356],
         ...,
         [-1.43035889,  0.11671448,  0.65223694, ...,  0.17622375,
          -1.28160095, -0.18078613],
         [ 0.53323364, -0.86508179,  1.15802002, ...,  1.27702332,
           0.74150085, -0.41879272],
         [ 1.24726868,  0.83074951, -0.21054077, ...,  0.503479  ,
           1.39602661,  0.44398499]],

        [[-0.86825562,  0.8348999 ,  0.59159851, ..., -0.04708862,
          -1.59819031,  0.13537598],
         [ 0.28744507, -1.1723938 ,  0.65242004, ...,  1.4127655 ,
           0.07455444, -1.59819031],
         [ 0.98696899,  0.40910339, -1.05075073, ..., -1.20281982,
           1.10862732,  0.53076172],
         ...,
         [ 1.23028564, -0.04708862

In [24]:
from syft import nn

In [25]:
nn.AvgPool2d(dp_tensor, 3).child.decode().shape

(50, 16, 1)

In [26]:
from torch.nn import AvgPool2d, MaxPool2d

In [27]:
from torch import Tensor
AvgPool2d(3)(Tensor(dp_tensor.child.decode())).shape

torch.Size([50, 16, 1])

In [28]:
nn.MaxPool2d(dp_tensor, kernel_size=2, stride=2).child.decode()

array([[[183.],
        [191.],
        [194.],
        ...,
        [205.],
        [195.],
        [190.]],

       [[207.],
        [199.],
        [199.],
        ...,
        [193.],
        [200.],
        [196.]],

       [[196.],
        [190.],
        [201.],
        ...,
        [188.],
        [228.],
        [183.]],

       ...,

       [[188.],
        [196.],
        [213.],
        ...,
        [202.],
        [199.],
        [227.]],

       [[195.],
        [195.],
        [192.],
        ...,
        [196.],
        [198.],
        [195.]],

       [[201.],
        [181.],
        [198.],
        ...,
        [188.],
        [169.],
        [230.]]])

In [29]:
MaxPool2d(kernel_size=2, stride=2)(Tensor(dp_tensor.child.decode()))

tensor([[[183.],
         [191.],
         [194.],
         ...,
         [205.],
         [195.],
         [190.]],

        [[207.],
         [199.],
         [199.],
         ...,
         [193.],
         [200.],
         [196.]],

        [[196.],
         [190.],
         [201.],
         ...,
         [188.],
         [228.],
         [183.]],

        ...,

        [[188.],
         [196.],
         [213.],
         ...,
         [202.],
         [199.],
         [227.]],

        [[195.],
         [195.],
         [192.],
         ...,
         [196.],
         [198.],
         [195.]],

        [[201.],
         [181.],
         [198.],
         ...,
         [188.],
         [169.],
         [230.]]])

In [30]:
torch.nn.functional.leaky_relu(Tensor(BatchNorm2d(dpt2, 3).child.decode()))

tensor([[[[ 8.6960e-02, -1.5196e-02,  1.0985e+00,  ...,  1.2473e+00,
            1.7622e-01, -1.1923e-02],
          [ 1.0985e+00,  5.9274e-01, -8.3533e-03,  ..., -1.7874e-02,
           -3.8905e-03, -2.9979e-03],
          [-1.8469e-02, -3.5930e-03,  2.9523e-01,  ...,  5.7205e-02,
           -1.7279e-02, -2.2736e-05],
          ...,
          [-1.4304e-02,  1.1671e-01,  6.5224e-01,  ...,  1.7622e-01,
           -1.2816e-02, -1.8079e-03],
          [ 5.3323e-01, -8.6508e-03,  1.1580e+00,  ...,  1.2770e+00,
            7.4150e-01, -4.1879e-03],
          [ 1.2473e+00,  8.3075e-01, -2.1054e-03,  ...,  5.0348e-01,
            1.3960e+00,  4.4398e-01]],

         [[-8.6826e-03,  8.3490e-01,  5.9160e-01,  ..., -4.7089e-04,
           -1.5982e-02,  1.3538e-01],
          [ 2.8745e-01, -1.1724e-02,  6.5242e-01,  ...,  1.4128e+00,
            7.4554e-02, -1.5982e-02],
          [ 9.8697e-01,  4.0910e-01, -1.0508e-02,  ..., -1.2028e-02,
            1.1086e+00,  5.3076e-01],
          ...,
     

In [31]:
nn.functional.leaky_ReLU(BatchNorm2d(dpt2, 3)).decode()

array([[[[ 8.69598389e-02, -1.51824951e-02,  1.09851074e+00, ...,
           1.24726868e+00,  1.76223755e-01, -1.19171143e-02],
         [ 1.09851074e+00,  5.92742920e-01, -8.34655762e-03, ...,
          -1.78680420e-02, -3.87573242e-03, -2.99072266e-03],
         [-1.84631348e-02, -3.58581543e-03,  2.95227051e-01, ...,
           5.72052002e-02, -1.72729492e-02, -1.52587891e-05],
         ...,
         [-1.42974854e-02,  1.16714478e-01,  6.52236938e-01, ...,
           1.76223755e-01, -1.28021240e-02, -1.80053711e-03],
         [ 5.33233643e-01, -8.63647461e-03,  1.15802002e+00, ...,
           1.27702332e+00,  7.41500854e-01, -4.18090820e-03],
         [ 1.24726868e+00,  8.30749512e-01, -2.09045410e-03, ...,
           5.03479004e-01,  1.39602661e+00,  4.43984985e-01]],

        [[-8.68225098e-03,  8.34899902e-01,  5.91598511e-01, ...,
          -4.57763672e-04, -1.59759521e-02,  1.35375977e-01],
         [ 2.87445068e-01, -1.17187500e-02,  6.52420044e-01, ...,
           1.41276550e